In [1]:
#基于双向LSTM的seq2seq字标注
#关键是有相应标注好的训练数据

In [3]:
# -*- coding:utf-8 -*-
import re
import numpy as np
import pandas as pd

word_size = 128
maxlen = 32
 
s = open('msr_train.txt').read().decode('gbk')
s = s.split('\r\n')
 
def clean(s): #整理一下数据，有些不规范的地方
    if u'“/s' not in s:
        return s.replace(u' ”/s', '')
    elif u'”/s' not in s:
        return s.replace(u'“/s ', '')
    elif u'‘/s' not in s:
        return s.replace(u' ’/s', '')
    elif u'’/s' not in s:
        return s.replace(u'‘/s ', '')
    else:
        return s
 
s = u''.join(map(clean, s))
s = re.split(u'[，。！？、]/[bems]', s)

In [5]:
data = [] #生成训练样本
label = []
def get_xy(s):
    s = re.findall('(.)/(.)', s)
    if s:
        s = np.array(s)
        return list(s[:,0]), list(s[:,1])
 
for i in s:
    x = get_xy(i)
    if x:
        data.append(x[0])
        label.append(x[1])
 
d = pd.DataFrame(index=range(len(data)))
d['data'] = data
d['label'] = label
d = d[d['data'].apply(len) <= maxlen]
d.index = range(len(d))
tag = pd.Series({'s':0, 'b':1, 'm':2, 'e':3, 'x':4})
 

In [6]:
chars = [] #统计所有字，跟每个字编号
for i in data:
    chars.extend(i)
 
chars = pd.Series(chars).value_counts()
chars[:] = range(1, len(chars)+1)
 
#生成适合模型输入的格式
from keras.utils import np_utils
d['x'] = d['data'].apply(lambda x: np.array(list(chars[x])+[0]*(maxlen-len(x))))
d['y'] = d['label'].apply(lambda x: np.array(map(lambda y:np_utils.to_categorical(y,5), tag[x].reshape((-1,1)))+[np.array([[0,0,0,0,1]])]*(maxlen-len(x))))
 

Using Theano backend.


In [7]:
#设计模型
word_size = 128
maxlen = 32
from keras.layers import Dense, Embedding, LSTM, TimeDistributed, Input, Bidirectional
from keras.models import Model
 
sequence = Input(shape=(maxlen,), dtype='int32')
embedded = Embedding(len(chars)+1, word_size, input_length=maxlen, mask_zero=True)(sequence)
blstm = Bidirectional(LSTM(64, return_sequences=True), merge_mode='sum')(embedded)
output = TimeDistributed(Dense(5, activation='softmax'))(blstm)
model = Model(input=sequence, output=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 
batch_size = 1024
history = model.fit(np.array(list(d['x'])), np.array(list(d['y'])).reshape((-1,maxlen,5)), batch_size=batch_size, nb_epoch=50)
 
#转移概率，单纯用了等概率
zy = {'be':0.5, 
      'bm':0.5, 
      'eb':0.5, 
      'es':0.5, 
      'me':0.5, 
      'mm':0.5,
      'sb':0.5, 
      'ss':0.5
     }
 
zy = {i:np.log(zy[i]) for i in zy.keys()}
 
def viterbi(nodes):
    paths = {'b':nodes[0]['b'], 's':nodes[0]['s']}
    for l in range(1,len(nodes)):
        paths_ = paths.copy()
        paths = {}
        for i in nodes[l].keys():
            nows = {}
            for j in paths_.keys():
                if j[-1]+i in zy.keys():
                    nows[j+i]= paths_[j]+nodes[l][i]+zy[j[-1]+i]
            k = np.argmax(nows.values())
            paths[nows.keys()[k]] = nows.values()[k]
    return paths.keys()[np.argmax(paths.values())]
 
def simple_cut(s):
    if s:
        r = model.predict(np.array([list(chars[list(s)])+[0]*(maxlen-len(s))]), verbose=False)[0][:len(s)]
        r = np.log(r)
        nodes = [dict(zip(['s','b','m','e'], i[:4])) for i in r]
        t = viterbi(nodes)
        words = []
        for i in range(len(s)):
            if t[i] in ['s', 'b']:
                words.append(s[i])
            else:
                words[-1] += s[i]
        return words
    else:
        return []
 
not_cuts = re.compile(u'([\da-zA-Z ]+)|[。，、？！\.\?,!]')
def cut_word(s):
    result = []
    j = 0
    for i in not_cuts.finditer(s):
        result.extend(simple_cut(s[j:i.start()]))
        result.append(s[i.start():i.end()])
        j = i.end()
    result.extend(simple_cut(s[j:]))
    return result

WARNING (theano.gof.compilelock): Overriding existing lock by dead process '823' (I am process '1221')


Epoch 1/50
289458/289458 [==============================] - 529s - loss: 0.7039 - acc: 0.2647   
Epoch 2/50
289458/289458 [==============================] - 524s - loss: 0.3903 - acc: 0.3122   
Epoch 3/50
289458/289458 [==============================] - 559s - loss: 0.3312 - acc: 0.3200   
Epoch 4/50
289458/289458 [==============================] - 532s - loss: 0.2963 - acc: 0.3247   
Epoch 5/50
289458/289458 [==============================] - 541s - loss: 0.2710 - acc: 0.3282   
Epoch 6/50
289458/289458 [==============================] - 540s - loss: 0.2508 - acc: 0.3309   
Epoch 7/50
289458/289458 [==============================] - 519s - loss: 0.2334 - acc: 0.3332   
Epoch 8/50
289458/289458 [==============================] - 566s - loss: 0.2186 - acc: 0.3351   
Epoch 9/50
289458/289458 [==============================] - 529s - loss: 0.2057 - acc: 0.3369   
Epoch 10/50
 77824/289458 [=======>......................] - ETA: 389s - loss: 0.1929 - acc: 0.3384

KeyboardInterrupt: 